<a href="https://colab.research.google.com/github/juligomez-web/Sys/blob/main/Ejercicio_1_6_y_1_7__Dashboard_en_Streamlit_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Ejercicio 1.6 y 1.7 — Dashboard en Streamlit**

**Indicaciones del monitor:**  
Solo se implementa el **dashboard** para los puntos **1.6 (Modulación)** y **1.7 (Calidad de la energía)**
El trabajo debe integrarse en un mismo entorno utilizando **Streamlit** con la estructura *MultiApp* del cuaderno `Demo_MultiApp_NoNgrok.ipynb`

---

###  **1.6 — Modulación (AM DSB-LC)**  
- Implementar la señal de **modulación en amplitud**:  
  $[
  s(t)=A_c\,[1+m\cos(2\pi f_m t)]\cos(2\pi f_c t)
  $]
- Permitir modificar:
  - Frecuencia de la portadora $(f_c$)
  - Frecuencia del mensaje $(f_m$)
  - Índice de modulación $(m$)
  - Amplitudes $(A_c$) y $(A_m$)
- Visualizar:
  - Señales **en el dominio del tiempo**
  - **Espectro** de frecuencias usando la **FFT**
- Analizar el efecto del índice de modulación $(m$) sobre las bandas laterales y la envolvente de la señal.

---

###  **1.7 — Calidad de la energía (THD y Factor de Potencia)**  
- Analizar un **rectificador de onda completa** con cargas:
  1. **Resistiva (R)**
  2. **RC en serie**
- Calcular la **Distorsión Total Armónica (THD)** a partir de la **FFT** de la señal de salida:
  $[
  THD=\frac{\sqrt{\sum_{n=2}^{N}V_n^2}}{V_1}
  $]
- Calcular el **factor de potencia por distorsión**:
  $[
  PF_{THD}=\frac{1}{\sqrt{1+THD^2}}
  $]
- Modelar la salida RC con:
  $[
  G(s)=\frac{1}{RCs+1}
  $]
- Permitir variar los valores de **R** y **C** para observar el efecto sobre el THD y el PF.  
- Mostrar en tablas y gráficas el impacto de la carga en la **calidad de la energía**.



In [ ]:
# ==== CREA LA ESTRUCTURA Y LOS ARCHIVOS DEL PROYECTO ====
import os
base = "/content/streamlit_multiapp"
apps = f"{base}/apps"
os.makedirs(apps, exist_ok=True)

# -------------------------
# requirements.txt
# -------------------------
open(f"{base}/requirements.txt","w").write("""streamlit==1.39.0
numpy
scipy
matplotlib
pandas
""")

# -------------------------
# multiapp.py  (patrón del demo)
# -------------------------
open(f"{base}/multiapp.py","w").write("""# Patrón MultiApp usado en el DEMO: contenedor de sub-aplicaciones Streamlit
# (idéntica idea al cuaderno Demo_MultiApp_NoNgrok.ipynb)
import streamlit as st

class MultiApp:
    def __init__(self):
        self.apps = []

    def add_app(self, title, func):
        # Registro de páginas/módulos
        self.apps.append({"title": title, "function": func})

    def run(self):
        # Navegación desde la barra lateral (como en el demo)
        st.sidebar.title("🧭 Navegación")
        app = st.sidebar.radio("Selecciona un módulo", self.apps, format_func=lambda a: a["title"])
        app["function"]()
""")

# -------------------------
# app.py  (lanzador principal, nombres 1.6/1.7)
# -------------------------
open(f"{base}/app.py","w").write("""# Lanzador principal (formato Demo_MultiApp_NoNgrok)
import streamlit as st
from multiapp import MultiApp
from apps import am_modulation, power_thd

st.set_page_config(page_title="Dashboard 1.6 y 1.7 (Colab, sin ngrok)", layout="wide")

st.title("📊 Dashboard — Taller (1.6 y 1.7)")
st.markdown(\"\"\"**Indicaciones del monitor:** implementar dashboard para los puntos **1.6 (Modulación)** y **1.7 (Calidad de la energía)**.

**Contenido:**
1) **1.6 — Modulación (AM DSB-LC)**
2) **1.7 — Calidad de la energía (THD, armónicos y PF por distorsión)**

> Navega desde la **barra lateral** (estructura MultiApp del demo).
\"\"\")

app = MultiApp()
app.add_app("1.6 — Modulación (AM)", am_modulation.app)
app.add_app("1.7 — Calidad de la energía (THD/PF)", power_thd.app)
app.run()
""")

# -------------------------
# apps/am_modulation.py   (punto 1.6)
# -------------------------
open(f"{apps}/am_modulation.py","w").write(r"""# Página 1 — 1.6 Modulación: AM (DSB-LC)
# Basada en el patrón del Demo_MultiApp_NoNgrok (MultiApp + navegación lateral)
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt

def am_signal(Ac, Am, fc, fm, m, t):
    # Señal del mensaje, portadora y AM DSB-LC
    mc = Am * np.cos(2*np.pi*fm*t)
    carrier = Ac * np.cos(2*np.pi*fc*t)
    s = (1 + m*np.cos(2*np.pi*fm*t)) * Ac * np.cos(2*np.pi*fc*t)
    return mc, carrier, s

def spectrum(x, Fs):
    # FFT con ventana de Hanning (mejora visual; no cambia la definición)
    N = len(x)
    X = np.fft.rfft(x * np.hanning(N))
    f = np.fft.rfftfreq(N, 1/Fs)
    mag = np.abs(X)/N
    return f, mag

def app():
    st.header("🎙️ 1.6 — Modulación: AM (DSB-LC)")
    st.caption("Página construida con el patrón MultiApp del Demo_MultiApp_NoNgrok.ipynb.")

    with st.expander("Resumen rápido (editable)", expanded=False):
        st.markdown(
            "- $s(t)=A_c\\,[1+m\\cos(2\\pi f_m t)]\\cos(2\\pi f_c t)$  \n"
            "- Índice de modulación ideal: $0\\le m\\le1$.  \n"
            "- En frecuencia aparecen $f_c$ y bandas laterales $f_c\\pm f_m$."
        )

    col1, col2 = st.columns(2)
    with col1:
        Fs = st.number_input("Frecuencia de registro [Hz]", min_value=10000, value=100000, step=5000)
        dur = st.slider("Duración [ms]", 10, 200, 50)/1000
        t = np.arange(0, dur, 1/Fs)

    with col2:
        fc = st.number_input("f_c (portadora) [Hz]", min_value=1000, value=10000, step=500)
        fm = st.number_input("f_m (mensaje) [Hz]",   min_value=10,   value=1000,  step=10)
        m  = st.slider("Índice de modulación m", 0.0, 1.2, 0.6, 0.05)

    Ac = st.number_input("Amplitud portadora Ac", min_value=0.1, value=1.0, step=0.1)
    Am = st.number_input("Amplitud mensaje (relativa)", min_value=0.0, value=1.0, step=0.1)

    m_sig, carrier, s = am_signal(Ac, Am, fc, fm, m, t)

    # Señales en el tiempo
    fig1 = plt.figure()
    plt.plot(t*1000, m_sig, label="m(t)")
    plt.plot(t*1000, carrier, label="c(t)")
    plt.plot(t*1000, s, label="s_AM(t)")
    plt.xlabel("Tiempo [ms]"); plt.ylabel("Amplitud")
    plt.title("Señales en el tiempo"); plt.grid(True); plt.legend()
    st.pyplot(fig1)

    # Espectro
    f, mag = spectrum(s, Fs)
    fig2 = plt.figure()
    plt.stem(f, mag, basefmt=" ")      # sin use_line_collection — Matplotlib reciente
    plt.xlim(0, fc + 3*fm)
    plt.xlabel("Frecuencia [Hz]"); plt.ylabel("|S(f)| (norm)")
    plt.title("Espectro (AM DSB-LC)"); plt.grid(True)
    st.pyplot(fig2)
""")

# -------------------------
# apps/power_thd.py   (punto 1.7)
# -------------------------
open(f"{apps}/power_thd.py","w").write(r"""# Página 2 — 1.7 Calidad de la energía: THD y PF (Rectificador)
# Sigue el flujo del cuaderno 3_Rectificador_RC_THD y la estructura del demo MultiApp.
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import pandas as pd

def onda_rectificada_completa(t, A, Fo):
    return A*np.abs(np.sin(2*np.pi*Fo*t))

def calcular_fft(x, Fs):
    # FFT con ventana de Hanning para espectro más limpio
    N = len(x)
    X = np.fft.rfft(x * np.hanning(N))
    f = np.fft.rfftfreq(N, 1/Fs)
    mag = np.abs(X)/N
    return f, mag

def calcular_thd(x, Fs, Ffund, n_harm=25):
    f, mag = calcular_fft(x, Fs)
    i1 = np.argmin(np.abs(f - Ffund))
    V1 = mag[i1] if mag[i1] != 0 else 1e-12
    acc = 0.0
    for k in range(2, n_harm+1):
        fk = k*Ffund
        if fk > Fs/2: break
        ik = np.argmin(np.abs(f - fk))
        acc += mag[ik]**2
    THD = np.sqrt(acc)/V1
    return THD, f, mag

def PF_from_THD(THD):
    # Factor de potencia por distorsión (definición usada en el cuaderno)
    return 1.0/np.sqrt(1 + THD**2)

def salida_rectificador_RC(R, C, Fo, A, n_periodos=5, Fs_factor=30):
    # Modelo de 1er orden (como en 3_Rectificador_RC_THD): G(s)=1/(RCs+1)
    Fs = Fs_factor*Fo
    t = np.arange(0, n_periodos*(1/Fo), 1/Fs)
    vin = onda_rectificada_completa(t, A, Fo)
    num = np.array([1.0]); den = np.array([R*C, 1.0])
    sistema = sig.TransferFunction(num, den)
    tout, vout, _ = sig.lsim(sistema, U=vin, T=t)
    return t, vin, vout, Fs

def app():
    st.header("⚡ 1.7 — Calidad de la energía: THD y Factor de Potencia")
    st.caption("Fundamental en 2·F0 para la rectificada. THD por FFT; PF por distorsión = 1/sqrt(1+THD^2).")

    with st.sidebar:
        Fo = st.number_input("Frecuencia de red F0 [Hz]", min_value=30, value=60, step=1)
        A  = st.number_input("Amplitud de entrada A", min_value=1.0, value=120.0, step=1.0)
        Fs_factor = st.slider("Factor de muestreo (Fs = k·F0)", 10, 80, 30)
        nper = st.slider("Nº de periodos a simular", 3, 12, 5)
        caso = st.radio("Tipo de carga", ["Resistiva (R)", "RC en serie"], index=1)
        R = st.number_input("R [Ω]", min_value=10.0, value=1000.0, step=10.0)
        C_uF = st.number_input("C [µF] (solo caso RC)", min_value=1.0, value=10.0, step=1.0)
        C = C_uF*1e-6

    Ffund = 2*Fo

    if caso.startswith("Resistiva"):
        Fs = Fs_factor*Fo
        t = np.arange(0, nper*(1/Fo), 1/Fs)
        vout = onda_rectificada_completa(t, A, Fo)
    else:
        t, _, vout, Fs = salida_rectificador_RC(R, C, Fo, A, n_periodos=nper, Fs_factor=Fs_factor)

    THD, f, mag = calcular_thd(vout, Fs, Ffund=Ffund)
    PF_THD = PF_from_THD(THD)

    # Métricas
    st.subheader("Resultados")
    c1, c2, c3 = st.columns(3)
    c1.metric("THD", f"{THD*100:.2f} %")
    c2.metric("PF por distorsión", f"{PF_THD:.3f}")
    c3.metric("Fundamental (rectificada)", f"{Ffund:.1f} Hz")

    # Señal temporal
    fig1 = plt.figure()
    plt.plot(t, vout, label="Salida v(t)")
    plt.xlabel("Tiempo [s]"); plt.ylabel("Voltaje [V]")
    plt.title(f"Rectificador con carga: {caso}")
    plt.grid(True); plt.legend()
    st.pyplot(fig1)

    # Espectro
    fig2 = plt.figure()
    plt.stem(f, mag, basefmt=" ")   # sin use_line_collection (compat Colab)
    plt.xlim(0, 1000 if Fo==60 else 20*Fo)
    plt.xlabel("Frecuencia [Hz]"); plt.ylabel("|V(f)| (norm)")
    plt.title("Espectro de magnitud"); plt.grid(True)
    st.pyplot(fig2)

    # Barrido simple R/C (apoyo al análisis)
    if caso.startswith("RC"):
        st.subheader("Barrido rápido de R y C")
        valores_R = [500, 1000, 2000]
        valores_C = [10e-6, 47e-6, 100e-6]
        data = []
        for Rb in valores_R:
            for Cb in valores_C:
                t_b, _, v_b, Fs_b = salida_rectificador_RC(Rb, Cb, Fo, A, n_periodos=nper, Fs_factor=Fs_factor)
                thd_b, _, _ = calcular_thd(v_b, Fs_b, Ffund)
                pf_b = PF_from_THD(thd_b)
                data.append([Rb, Cb*1e6, thd_b*100, pf_b])
        df = pd.DataFrame(data, columns=["R [Ω]", "C [µF]", "THD [%]", "PF_THD"])
        st.dataframe(df, use_container_width=True)
""")

print("✅ Archivos creados en:", base)


✅ Archivos creados en: /content/streamlit_multiapp


In [ ]:
%cd /content/streamlit_multiapp
!pip -q install -r requirements.txt


/content/streamlit_multiapp


In [ ]:
# 1) cierra procesos anteriores y limpia log
!pkill -f "streamlit run" || true
!rm -f /content/streamlit.log

# 2) entra a la carpeta del proyecto
%cd /content/streamlit_multiapp

# 3) lanza streamlit en segundo plano (flags compatibles con Colab)
!STREAMLIT_BROWSER_GATHER_USAGE_STATS=false \
  streamlit run app.py \
  --server.port 8501 \
  --server.headless true \
  --server.address 0.0.0.0 \
  --server.enableCORS false \
  --server.enableXsrfProtection false \
  --server.fileWatcherType none \
  &> /content/streamlit.log &

import time
time.sleep(6)

# 4) muestra inicio del log (confirmación)
print(open('/content/streamlit.log').read()[:800])


^C
/content/streamlit_multiapp

  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501




In [ ]:
# A) Enlace del proxy de Colab
from google.colab import output
url = output.eval_js("google.colab.kernel.proxyPort(8501)")
print("🔗 Abre tu dashboard aquí:", url)


🔗 Abre tu dashboard aquí: https://8501-m-s-3h60syiibm24r-d.us-east1-1.prod.colab.dev
